## Drafts for getting Additional Drug properties

Drafts for getting Additional Drug properties

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")


import os
import pubchempy as pcp
import re
from pubchempy import Compound

In [2]:
os.listdir("results")

['drug_features_with_properties.csv',
 'merged_drug_profiles_sigmoid4_23.csv',
 'drug_features_pubchem.csv',
 'filtered_drug_profiles_13.csv',
 'filtered_drug_profiles_12.csv',
 'filtered_drug_profiles_23.csv',
 'filtered_drug_profiles.csv',
 'merged_drug_profiles_sigmoid4_123.csv',
 'filtered_drug_profiles_123.csv']

In [3]:
df123= pd.read_csv("results/merged_drug_profiles_sigmoid4_123.csv")
df23= pd.read_csv("results/merged_drug_profiles_sigmoid4_23.csv")
drug_features = pd.read_csv('data/Drug_Features.csv').rename(columns={"Drug ID": "DRUG_ID", 
                                                                      "Drug Name" : "Drug_Name",
                                                                      "Target Pathway": "Target_Pathway"})

In [4]:
drugs_123 = df123["DRUG_ID"].unique()
drugs_23 = df23["DRUG_ID"].unique()
drugs = drug_features["DRUG_ID"].unique()
print("Number of all available drugs:", len(drugs))
print("Number of drugs in filtered dataset with filtering scenario [2,3]:", len(drugs_23))
print("Number of drugs in filtered dataset with filtering scenario [1,2,3]:", len(drugs_123))
print("Number of drugs in both filterd datasets:", len(set(drugs_123)&set(drugs_23)))

Number of all available drugs: 265
Number of drugs in filtered dataset with filtering scenario [2,3]: 145
Number of drugs in filtered dataset with filtering scenario [1,2,3]: 145
Number of drugs in both filterd datasets: 145


In [5]:
drug_features.head()

,DRUG_ID,Drug_Name,Synonyms,Target,Target_Pathway
0,1,Erlotinib,"Tarceva, RG-1415, CP-358774, OSI-774, Ro-50823...",EGFR,EGFR signaling
1,3,Rapamycin,"AY-22989, Sirolimus, WY-090217, Torisel, Rapamune",MTORC1,PI3K/MTOR signaling
2,5,Sunitinib,"Sutent, Sunitinib Malate, SU-11248","PDGFR, KIT, VEGFR, FLT3, RET, CSF1R",RTK signaling
3,6,PHA-665752,"PHA665752, PHA 665752",MET,RTK signaling
4,9,MG-132,"LLL cpd, MG 132, MG132","Proteasome, CAPN1",Protein stability and degradation


### get drug features from pubchempy

In [7]:
# how it works
n=0
drug_id = drug_features["DRUG_ID"][n]
drug_name = drug_features["Drug_Name"][n]
pcp_id = pcp.get_compounds(drug_name, 'name')
print(drug_name, ": PubChem_ID:", re.search(r'\((.*?)\)', str(pcp_id)).group(1))

ids = pcp.get_compounds("Obatoclax Mesylate", 'name')
print("\nObatoclax Mesylate:", re.findall(r'\((.*?)\)', str(ids)))

Erlotinib : PubChem_ID: 176870

Obatoclax Mesylate: ['16681698', '46930996', '16727411', '70700268', '71816154', '56843157', '122362431']


In [8]:
%%time

for drug_id in drug_features["DRUG_ID"].unique():
    drug_index = drug_features[drug_features["DRUG_ID"]==drug_id].index
    drug_name = drug_features.loc[drug_index, "Drug_Name"].values[0]
    deriv = pcp.get_compounds(drug_name, 'name')
    drug_features.loc[drug_index, "deriv_found"] = len(deriv)
    try:
        drug_features.loc[drug_index, "PubChem_ID"]= re.findall(r'\((.*?)\)', str(deriv))
    except:
        if len(deriv)>1:
            drug_features.loc[drug_index, "PubChem_ID"]= str([np.int(x) for x in re.findall(r'\((.*?)\)', str(deriv))]).strip("[").strip("]")
        else:
            drug_features.loc[drug_index, "PubChem_ID"]= 0

CPU times: user 6.89 s, sys: 432 ms, total: 7.32 s
Wall time: 3min 9s


In [9]:
drug_features["deriv_found"].value_counts()

1.0     215
0.0      38
2.0       3
16.0      2
3.0       1
19.0      1
8.0       1
5.0       1
7.0       1
6.0       1
13.0      1
Name: deriv_found, dtype: int64

In [10]:
# check parsing pubchem_ids for drugs in filtered datasets
drug_features.set_index("DRUG_ID").loc[drugs_123, "deriv_found"].value_counts()

1.0     120
0.0      16
2.0       2
19.0      1
3.0       1
5.0       1
6.0       1
13.0      1
7.0       1
8.0       1
Name: deriv_found, dtype: int64

**Conclusion:** For 15 drugs there is no directly found matching of compound name and its pubchem_id,
    for 7 drugs more than 3 matching was found

In [11]:
drug_features[drug_features["deriv_found"]>1]

,DRUG_ID,Drug_Name,Synonyms,Target,Target_Pathway,deriv_found,PubChem_ID
5,11,Paclitaxel,"BMS-181339-01, Taxol, Onxol, Paxene, Praxel, A...",Microtubule stabiliser,Mitosis,13.0,"36314, 441276, 4666, 23636632, 44155032, 73759..."
32,89,Parthenolide,-,HDAC1,Chromatin histone acetylation,16.0,"7251185, 6473881, 5420804, 5420805, 4692, 1080..."
45,140,Vinorelbine,"vinorelbine tartrate, Navelbine, Exelbine",Microtubule destabiliser,Mitosis,6.0,"5311497, 60780, 13200033, 5672, 44424639, 7175..."
72,182,Obatoclax Mesylate,"GX15-070MS, Obatoclax, GX15-070","BCL2, BCL-XL, BCL-W, MCL1",Apoptosis regulation,7.0,"16681698, 46930996, 16727411, 70700268, 718161..."
76,190,Bleomycin,-,dsDNA break induction,DNA replication,5.0,"5360373, 84068, 5460769, 456190, 72467"
81,197,Bryostatin 1,Bryostatin,PKC,"Other, kinases",16.0,"5280757, 433902, 22524140, 44277532, 5458769, ..."
86,203,BMS-345541,"BMS345541, IKK Inhibitor 3","IKK1, IKK2","Other, kinases",2.0,"9813758, 9926054"
153,309,Y-39983,-,ROCK,Cytoskeleton,2.0,"11507964, 9810884"
169,1004,Vinblastine,Velban,Microtubule destabiliser,Mitosis,8.0,"241903, 13342, 3823887, 6710780, 44417235, 111..."
181,1016,Temsirolimus,"CCI-779, Torisel",MTOR,PI3K/MTOR signaling,19.0,"6918289, 18293306, 23724530, 148191, 53486199,..."


### Manual corrections

In [12]:
# error in spelling
# https://www.cancerrxgene.org/compounds
# in the data:Lestauritinib, true name: Lestaurtinib (difference in one letter)
error_names_dict={"Lestauritinib": "Lestaurtinib"}
error_name = "Lestauritinib"

# correct the search results
error_drug_index = drug_features[drug_features["Drug_Name"]==error_name].index
correct_drug_name = error_names_dict[error_name]
drug_features.loc[error_drug_index, "Drug_Name"] = correct_drug_name

In [13]:
new_synonyms = {"Y-39983": {"Synonyms": "Y-33075",
                           "reference": ["https://www.medchemexpress.com/Y-33075.html",
                            "https://www.nature.com/articles/s41467-019-13781-3"]}}

manual_corrections = {
    "Lestaurtinib":{"pubchem_id" : 126565,
               "reference" : "https://www.cancerrxgene.org/compounds"},
    
    "WZ-1-84": {"pubchem_id" : 49821040,
               "reference" : "http://lincs.hms.harvard.edu/db/datasets/20119/smallmolecules"},
    
    "GW441756": {"pubchem_id" : 9943465 ,
               "reference" : "",
               "note": "no result in drugbank"},
    
    "Parthenolide" : {"pubchem_id" : 6473881,
               "reference" : "https://www.drugbank.ca/drugs/DB13063"},
    
    "Obatoclax Mesylate": {"pubchem_id" : 347828476,
               "reference" : "https://www.drugbank.ca/drugs/DB12191"},
    
    "Bleomycine": {"pubchem_id" : 72467,
               "reference" : "https://www.drugbank.ca/drugs/DB00290"},
    
    "Y-39983": {"pubchem_id" : 20601328,
               "reference" : "https://www.medchemexpress.com/Y-33075.html"},
    
    "JW-7-52-1": {"pubchem_id" : 20822503,
               "reference" : "https://pharmacodb.ca/drugs/392"},
    
    "VNLG/124": { "pubchem_id": 24894414, 
                  "reference": "https://www.cancerrxgene.org/compounds" },
    
    "PDK1 inhibitor 7": { "pubchem_id": 56965967, 
                         "reference": "https://www.cancerrxgene.org/compounds"},
    
    "KIN001-260": {"pubchem_id": 10451420, 
                   "reference": "https://www.cancerrxgene.org/compounds"},
    
    "SB52334": {"pubchem_id": 9967941, 
                "reference": "https://www.cancerrxgene.org/compounds"},
    
    "KIN001-270": { "pubchem_id": 66577006, 
                   "reference": "https://www.cancerrxgene.org/compounds"},
    
    "Cisplatin": {"pubchem_id": 84691, 
                  "reference": "https://www.cancerrxgene.org/compounds"},
    
    "Cetuximab": {"pubchem_id": 85668777, 
                  "reference": "https://www.cancerrxgene.org/compounds"},
    
    "Nutlin-3a (-)": { "pubchem_id": 
                      11433190, "reference": ""},
    
    "681640": { "pubchem_id": 10384072, 
               "reference": ""},
    
    "MPS-1-IN-1": {"pubchem_id": 25195352, 
                   "reference": ""},
    
    "KIN001-266": { "pubchem_id": 44143370, 
                   "reference": ""},
    
    "JW-7-52-1" : {"pubchem_id": 49836027, 
                   "reference": ""},
    
    "Vinorelbine": {"pubchem_id": 44424639, 
                   "reference": "https://www.drugbank.ca/drugs/DB00361"},
    
    "Paclitaxel": {"pubchem_id": 36314, 
                   "reference": "https://www.drugbank.ca/drugs/DB01229"},
    
    "Bleomycin": {"pubchem_id": 5360373, 
                   "reference": "https://www.drugbank.ca/drugs/DB00290"},
    
    "Vinblastine": {"pubchem_id": 13342, 
                   "reference": "https://www.drugbank.ca/drugs/DB00570"},
    
    
    "THZ-2-102-1" : {"pubchem_id": 146011539, 
                   "reference": "Katjusa Koler's suggestion"},
    
    "THZ-2-49" : {"pubchem_id": 78357763 , 
                   "reference": ["https://www.cancerrxgene.org/compounds", 
                                "https://www.medchemexpress.com/THZ2.html",
                                "https://pubchem.ncbi.nlm.nih.gov/compound/78357763"]},
    
    "QL-XII-47": {"pubchem_id": 71748056, 
                   "reference": "https://lincs.hms.harvard.edu/db/sm/10077-101-1/"},
    
    "BMS-345541" : {"pubchem_id": 9813758, 
                   "reference": ""},
    
    "Temsirolimus" : {"pubchem_id": 23724530, 
                   "reference": "https://www.drugbank.ca/drugs/DB06287"},
    
    "SB590885" : {"pubchem_id": 135398506, 
                   "reference": "https://pubchem.ncbi.nlm.nih.gov/#query=SB590885"},
    
    "WZ3105" : {"pubchem_id": 42628507, 
                   "reference": "https://lincs.hms.harvard.edu/db/sm/10084-101/"},
    
    "NPK76-II-72-1" : {"pubchem_id": 46843648, 
                   "reference": "https://lincs.hms.harvard.edu/db/sm/10070-101/"},
    
    "JW-7-24-1" : {"pubchem_id": 69923936, 
                   "reference": "https://lincs.hms.harvard.edu/db/sm/10019-101/"},
    "Bryostatin 1" : {"pubchem_id": 6435419, 
                   "reference": "https://pubchem.ncbi.nlm.nih.gov/#query=Bryostatin%201"},
    "QL-XI-92": {"pubchem_id": "73265214",
                 "reference": "Katjusa Koler's & Dennis Wang's database"}
        
    }
    

# Vinorelbine: pubchem_id 46507772, DrugBank DB00361 (PubChem Substance)
# Vinorelbine Ditartarate:  pubchem_id 44424639 PubChem Compound  ***

# Paclitaxel: PubChem Compound 36314 ***, PubChem Substance 46506910

# Bleomycin PubChem Compound 5360373 *** PubChem Substance 46509116

# Vinblastine PubChem Compound 13342 *** PubChem Substance 46504550

# THZ-2-102-1 https://www.cancerrxgene.org/compounds PubCHEM_Id: None
# THZ-2-49 https://www.cancerrxgene.org/compounds PubCHEM_Id: None
# https://www.medchemexpress.com/THZ2.html - CAS1604810-84-5 PubCHEM_Id 78357763 THZ2
# https://pubchem.ncbi.nlm.nih.gov/compound/78357763
# https://pharmacodb.ca/drugs/687

# "QL-XII-47": https://lincs.hms.harvard.edu/db/sm/10077-101-1/ "PubCHEM_Id": 71748056, 
# https://pubchem.ncbi.nlm.nih.gov/compound/71748056  - QL47

# BMS-345541 PubCHEM_Id 9813758

# Temsirolimus PubChem Compound 23724530 *** PubChem Substance 99443243
#https://www.cancerrxgene.org/compounds	6918289	???

# SB590885 (E) PubCHEM_Id 135398506  or (Z) PubCHEM_Id 135421339 

In [9]:
corrections_pubchem_id = {
    "Temsirolimus": 6918289,
    "Vinorelbine": 5311497,
    "Y-39983": 9810884,
    "GW441756": 9943465, 
    "Vinblastine": 6710780,
    "Bryostatin 1": 5280757,
    "Parthenolide": 7251185,
    "Obatoclax Mesylate": 11404337,
    "Bleomycin (50 uM)": 5460769,
    "SB590885": 11316960,
    "Paclitaxel" :36314,
    "BMS-345541": 9813758 
}

# BMS-345541 https://www.cancerrxgene.org/compounds 9813758 *** and 9926054

### Manual corrections

In [ ]:
for drug_name in manual_corrections:
    drug_index = drug_features[drug_features["Drug_Name"]==drug_name].index
    drug_features.loc[drug_index, "deriv_found"] = 1
    drug_features.loc[drug_index, "PubChem_ID"]= corrections_pubchem_id[drug_name]

# more_corrections
for drug_name in corrections_pubchem_id:
    drug_index = drug_features[drug_features["Drug_Name"]==drug_name].index
    drug_features.loc[drug_index, "deriv_found"] = 1
    drug_features.loc[drug_index, "PubChem_ID"]= corrections_pubchem_id[drug_name]

In [15]:
# check parsing pubchem_ids for drugs in filtered datasets
drug_features.set_index("DRUG_ID").loc[drugs_123, "deriv_found"].value_counts()

1.0    139
0.0      6
Name: deriv_found, dtype: int64

**Conclusion:** Problem with 19 drugs
Need to find how important these drugs are

In [16]:
cell_features = pd.read_csv("data/Cell_Line_Features_PANCAN_simple_MOBEM.tsv", sep="\t")

stat_data123 = df123.groupby(["DRUG_ID"])[["COSMIC_ID"]].count().rename(columns={"COSMIC_ID": "count_cell_lines"})\
            .sort_values("count_cell_lines", ascending=False)
stat_data23 = df23.groupby(["DRUG_ID"])[["COSMIC_ID"]].count().rename(columns={"COSMIC_ID": "count_cell_lines"})\
            .sort_values("count_cell_lines", ascending=False)

statistics123 = pd.merge(left = stat_data123, right = drug_features, how= "left", on = "DRUG_ID").sort_values("count_cell_lines", ascending =False)
statistics23 = pd.merge(left = stat_data23, right = drug_features, how= "left", on = "DRUG_ID").sort_values("count_cell_lines", ascending =False)

In [17]:
statistics23[statistics23["deriv_found"]!=1.0]

,DRUG_ID,count_cell_lines,Drug_Name,Synonyms,Target,Target_Pathway,deriv_found,PubChem_ID
63,164,13,JQ12,-,"HDAC1, HDAC2",Chromatin histone acetylation,0.0,0
77,225,7,Genentech Cpd 10,-,"AURKA, AURKB",Mitosis,0.0,0
94,330,4,XMD13-2,-,RIPK1,Apoptosis regulation,0.0,0
113,1037,2,BX796,BX-796,"TBK1, PDK1 (PDPK1), IKK, AURKB, AURKC",Other,0.0,0
138,329,1,QL-XI-92,-,DDR1,Other,0.0,0
133,211,1,TL-2-105,-,not defined,ERK MAPK signaling,0.0,0


In [13]:
drug_features.to_csv("results/drug_features_pubchem.csv")

### Extracting properties from PubChem

In [19]:
c = Compound.from_cid(176870)
c.elements.count("O"), c.elements.count("C"), c.elements.count("N")

(4, 22, 3)

In [20]:
%%time
for i, PubChem_id in tqdm(list(enumerate(drug_features["PubChem_ID"].values))):
# for PubChem_id in drug_features["PubChem_ID"].values[:5]:
    try:
        drug_index = drug_features[drug_features["PubChem_ID"]==PubChem_id].index
        
        c = Compound.from_cid(PubChem_id)
        
        drug_features.loc[drug_index, "molecular_weight"] = c.molecular_weight
   
        drug_features.loc[drug_index, "elements"] = str(set(c.elements)).strip("{").strip("}")
        
        bonds = [int(str(i).split(",")[-1].strip(")")) for i in c.bonds]
        drug_features.loc[drug_index, "2bonds"] = bonds.count(2)
        drug_features.loc[drug_index, "3bonds"] = bonds.count(3)

        drug_features.loc[drug_index, "xlogp"] = c.xlogp
        drug_features.loc[drug_index, "formal_charge"] = c.charge
    
        drug_features.loc[drug_index, "surface_area"] = c.tpsa

        drug_features.loc[drug_index, "complexity"] = c.complexity

        drug_features.loc[drug_index, "complexity"] = c.h_bond_donor_count

        drug_features.loc[drug_index, "h_bond_acceptor_count"] = c.h_bond_acceptor_count

        drug_features.loc[drug_index, "rotatable_bond_count"] = c.rotatable_bond_count

        drug_features.loc[drug_index, "heavy_atom_count"] = c.heavy_atom_count

        drug_features.loc[drug_index, "atom_stereo_count"] = c.atom_stereo_count

        drug_features.loc[drug_index, "defined_atom_stereo_count"] = c.defined_atom_stereo_count

        drug_features.loc[drug_index, "undefined_atom_stereo_count"] = c.undefined_atom_stereo_count

        drug_features.loc[drug_index, "bond_stereo_count"] = c.bond_stereo_count

        drug_features.loc[drug_index, "covalent_unit_count"] = c.covalent_unit_count
        drug_features.loc[drug_index, "molecular_formula"] = c.molecular_formula

        drug_features.loc[drug_index, "canonical_smiles"] = c.canonical_smiles

        drug_features.loc[drug_index, "inchi_string"] = c.inchi

        drug_features.loc[drug_index, "inchi_key"] = c.inchikey
    except:
        pass

# # fingerprint
# # Raw padded and hex-encoded fingerprint, as returned by the PUG REST API.

# # cactvs_fingerprint
# # PubChem CACTVS fingerprint.
# # Each bit in the fingerprint represents the presence or absence of one of 881 chemical substructures.
# # More information at ftp://ftp.ncbi.nlm.nih.gov/pubchem/specifications/pubchem_fingerprints.txt


100%|██████████| 265/265 [02:46<00:00,  1.59it/s]

CPU times: user 11.2 s, sys: 582 ms, total: 11.8 s
Wall time: 2min 46s


### Additional features from splitting columns in drug_features

In this section, we are going to have some dumnies columns for Target and Target_Pathway
As in Target column only several drugs have more than 3 values, only three dumny columns were created

Converting of Target Pathway resulted in 29 new columns

It is also worth considering elements columns and that deleting columns with C and H which are present in all the compounds


In [21]:
drug_features["first_target"] = drug_features["Target"].str.split(",", expand=True).fillna(0)[0]
drug_features["second_target"] = drug_features["Target"].str.split(",", expand=True).fillna(0)[1]
drug_features["third_target"] = drug_features["Target"].str.split(",", expand=True).fillna(0)[2]

#### dumnies columns for Target Pathway (29 values)

In [22]:
drug_features = pd.concat([drug_features, pd.get_dummies(drug_features["Target_Pathway"])], axis=1).drop("Target_Pathway", axis=1)
drug_features.shape

(265, 52)

In [103]:
drug_features = pd.read_csv("results/drug_features_with_properties.csv")
drug_features.shape

(265, 53)

In [104]:
all_elements = list(set(drug_features["elements"].str.split(",", expand=True).fillna(0).values.flatten())- set([0," 'C'", "'C'", " 'H'"]))
print(len(all_elements))
elements_in_drugs= [atom.strip(" ").strip("'") for atom in all_elements]
elements_in_drugs

11


['O', 'Cl', 'F', 'S', 'Br', 'O', 'Pt', 'P', 'B', 'N', 'I']

In [112]:
%%time
for drug_index in drug_features.index:
    compound_elements = drug_features.loc[drug_index, "elements"]
    try:
        for i, atom in list(enumerate(elements_in_drugs)):
            if all_elements[i] in compound_elements:
                drug_features.loc[drug_index, atom] = 1
            else:
                drug_features.loc[drug_index, atom] = 0
    except:
        print(drug_index)
        drug_features.loc[drug_index, atom] = 0

58
72
92
98
109
116
134
135
158
159
161
199
225
226
228
229
231
237
248
257
CPU times: user 1.38 s, sys: 77.5 ms, total: 1.46 s
Wall time: 1.41 s


In [116]:
display(drug_features.loc[[58,59,72], :].T)

,58,59,72
Unnamed: 0,58,59,72
DRUG_ID,164,165,182
Drug_Name,JQ12,DMOG,Obatoclax Mesylate
Synonyms,-,Dimethyloxalylglcine,"GX15-070MS, Obatoclax, GX15-070"
Target,"HDAC1, HDAC2",HIF-PH,"BCL2, BCL-XL, BCL-W, MCL1"
...,...,...,...
B,NaN,0,NaN
Br,NaN,0,NaN
Pt,NaN,0,NaN
P,NaN,0,NaN


In [41]:
drug_features.drop("Unnamed: 0", axis=1).to_csv("results/drug_features_with_properties2.csv")

### Features from DrugBank.ca (data scapping in other script)

In [24]:
os.listdir("drugs")

['293_Amuvatinib_Ingredient_1.csv',
 '272_AR-42_Property_1.csv',
 '1016_Temsirolimus_Property_1.csv',
 '1502_Bicalutamide_Property_3.csv',
 '179_5-Fluorouracil_Property_2.csv',
 '136_Mitomycin-C_Property_2.csv',
 '1036_PLX-4720_Property_1.csv',
 '1054_Palbociclib_Property_2.csv',
 '140_Vinorelbine_Ingredient_1.csv',
 '1_Erlotinib_Property_6.csv',
 '155_Ponatinib_Property_3.csv',
 '1199_Tamoxifen_Property_8.csv',
 '150_Bicalutamide_Property_2.csv',
 '1372_Trametinib_Target_1.csv',
 '150_Bicalutamide_Property_3.csv',
 '1199_Tamoxifen_Property_9.csv',
 '1003_Camptothecin_Property_4.csv',
 '155_Ponatinib_Property_2.csv',
 '1371_PLX-4720_Target_1.csv',
 '1_Erlotinib_Property_7.csv',
 '293_Amuvatinib_Target_1.csv',
 '186_Bexarotene_Property_6.csv',
 '229_Enzastaurin_Property_1.csv',
 '341_Selisistat_Property_1.csv',
 '136_Mitomycin-C_Property_3.csv',
 '119_Lapatinib_Property_6.csv',
 '179_5-Fluorouracil_Property_3.csv',
 '254_Quizartinib_Property_1.csv',
 '1502_Bicalutamide_Property_2.csv',


In [31]:
%%time
drugs = {}
no_data =[]
for drug_id in drugs_123:
    drug_name = drug_features[drug_features["DRUG_ID"]==drug_id]["Drug_Name"].values[0]
    file_name = "drugs/*" + drug_name + "*"
    files = glob.glob(file_name)
    drugs[drug_name] = files
    if len(files)==0:
        no_data.append(drug_id)

CPU times: user 253 ms, sys: 103 ms, total: 356 ms
Wall time: 357 ms


In [32]:
print("Number of drugs in filtered dataset with filtering scenario [1,2,3]:", len(drugs_123))
print("Number of drugs from filtered dataset found in DrugBank:", len(drugs_123) - len(no_data))

Number of drugs in filtered dataset with filtering scenario [1,2,3]: 145
Number of drugs from filtered dataset found in DrugBank: 63


### Check presence of drug properties for the most important drugs (which have more than 50 records)

In [29]:
merged_123 = pd.read_csv("results/merged_drug_profiles_sigmoid4_123.csv")
stat = merged_123.groupby("DRUG_ID")[["COSMIC_ID"]].count()
most_important_drugs = stat[stat["COSMIC_ID"]>50].index

drugs = {}
no_data =[]
for drug_id in most_important_drugs:
    drug_name = drug_features[drug_features["DRUG_ID"]==drug_id]["Drug_Name"].values[0]
    file_name = "drugs/*" + drug_name + "*"
    files = glob.glob(file_name)
    drugs[drug_name] = files
    if len(files)==0:
        no_data.append(drug_id)
print("Number of most important drugs:", len(most_important_drugs))
print("Number of drugs from filtered datasetfound in DrugBank:", len(most_important_drugs) - len(no_data))

Number of most important drugs: 11
Number of drugs from filtered datasetfound in DrugBank: 4


In [43]:
d1 = pd.read_csv("results/drug_features_with_properties.csv")
#with corrected Pubchem ids
d2 = pd.read_csv("results/drug_features_with_properties2.csv")

drug_name = "Temsirolimus"
d = pd.DataFrame()
d["d1"] = np.hstack(d1[d1["Drug_Name"]==drug_name].T.values)
d["d2"] = np.hstack(d2[d2["Drug_Name"]==drug_name].T.values)
d

,d1,d2
0,181,181
1,1016,1016
2,Temsirolimus,Temsirolimus
3,"CCI-779, Torisel","CCI-779, Torisel"
4,MTOR,MTOR
5,1,1
6,23724530,6918289
7,1030.3,1030.3
8,"'C', 'N', 'O', 'H'","'C', 'N', 'O', 'H'"
9,10,10


In [44]:
d1.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
DRUG_ID,1,3,5,6,9
Drug_Name,Erlotinib,Rapamycin,Sunitinib,PHA-665752,MG-132
Synonyms,"Tarceva, RG-1415, CP-358774, OSI-774, Ro-50823...","AY-22989, Sirolimus, WY-090217, Torisel, Rapamune","Sutent, Sunitinib Malate, SU-11248","PHA665752, PHA 665752","LLL cpd, MG 132, MG132"
Target,EGFR,MTORC1,"PDGFR, KIT, VEGFR, FLT3, RET, CSF1R",MET,"Proteasome, CAPN1"
deriv_found,1,1,1,1,1
PubChem_ID,176870,5284616,5329102,10461815,462382
molecular_weight,393.4,914.2,398.5,641.6,475.6
elements,"'C', 'N', 'O', 'H'","'C', 'N', 'O', 'H'","'O', 'F', 'N', 'H', 'C'","'Cl', 'O', 'S', 'N', 'H', 'C'","'C', 'N', 'O', 'H'"
2bonds,8,9,8,13,7
